## Web-Scraping

In [47]:
# Dependencies
from bs4 import BeautifulSoup as bs

import requests
import pymongo
from datetime import datetime as dt
import pandas as pd
from splinter import Browser

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [22]:
# URL of page to be scraped.  Pass URL into splinter browser
mars_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_url)

In [23]:
# Scrape URL and create BeautifulSoup object
html = browser.html
soup = bs(html, 'html.parser')

# Print formatted version of the soup
# print(soup.prettify())

# Find the current news titles 
titles = soup.find_all('div', class_="content_title")

# Find the news paragraphs
paras = soup.find_all('div', class_="article_teaser_body")

# Assign the text to variables with text only
news_title = titles[1].text
news_p = paras[1].text

print(news_title)
print(news_p)

MOXIE Could Help Future Rockets Launch Off Mars
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.


### JPL Mars Space Images

In [25]:
# URL of page to be scraped.  Pass URL into splinter browser
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [63]:
# Create BeautifulSoup object
jpl_html = browser.html
soup = bs(jpl_html, 'html.parser')

# Find the featured image from home page and click Full Image button
browser.links.find_by_partial_text('Full Image')

# Find the more info button and click
browser.links.find_by_partial_text('more info')

# Use BeautifulSoup to find the full size .jpg image URL
for link in soup.find_all('figure', class_='lede'):
    jpl_img = link.find('a').get('href')
#     print(jpl_img)

jpl_base = 'https://www.jpl.nasa.gov'
feature_image_url = jpl_base + jpl_img
print(feature_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19041_hires.jpg


### Mars Facts

In [ ]:
facts_url = https://space-facts.com/mars/